In [31]:
# fromt __future__ import absolute_import, division, print_function
# vectorization
import codecs
# regex
import glob
# concurrency
import multiprocessing
# operating system
import os
# pretty printing , human readable
import pprint

# regular expression
import re
# natural language tool kit
import nltk

# word to vector
import gensim.models.word2vec as wtv
# dimensionality reduction

import sklearn.manifold 

# math
import numpy as np
# data 
import pandas as pd
# data visualization
import matplotlib.pyplot as plt

import seaborn as sns


In [32]:
nltk.download('punkt') #pretrained tokenizer
nltk.download('stopwords') #stopwords 


[nltk_data] Downloading package punkt to /home/shubhamsks/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/shubhamsks/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [33]:
# get the bookfile names matching txt file
book_filenames = sorted(glob.glob('./*.txt'))
book_filenames

['./got1.txt', './got2.txt', './got3.txt', './got4.txt', './got5.txt']

In [34]:
f = open(book_filenames[1],'r')


In [35]:
# Corpus
corpus_raw = u""
for fname in book_filenames:
    print('Reading {0} ...'.format(fname))
    with codecs.open(fname,'r','utf-8') as bookfile:
        corpus_raw += bookfile.read()
    print('Corpus  is now {0} character long '.format(len(corpus_raw)))
    print()

Reading ./got1.txt ...
Corpus  is now 1770659 character long 

Reading ./got2.txt ...
Corpus  is now 4071041 character long 

Reading ./got3.txt ...
Corpus  is now 6391405 character long 

Reading ./got4.txt ...
Corpus  is now 8107945 character long 

Reading ./got5.txt ...
Corpus  is now 9719485 character long 



In [36]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [37]:
raw_sentences = tokenizer.tokenize(corpus_raw)


In [38]:
def sentence_to_word_list(raw):
    clean = re.sub("[^a-zA-Z]", " ", raw)
    words = clean.split()
    return words

In [41]:
# sentences where each word is tokenized
sentences = []
for raw_sentence in raw_sentences:
    if len(raw_sentence) > 0:
        sentences.append(sentence_to_word_list(raw_sentence))

In [43]:
print(sentences[:100])

[['This', 'edition', 'contains', 'the', 'complete', 'text', 'of', 'the', 'original', 'hardcover', 'edition'], ['NOT', 'ONE', 'WORD', 'HAS', 'BEEN', 'OMITTED'], ['A', 'CLASH', 'OF', 'KINGS', 'A', 'Bantam', 'Spectra', 'Book', 'PUBLISHING', 'HISTORY', 'Bantam', 'Spectra', 'hardcover', 'edition', 'published', 'February', 'Bantam', 'Spectra', 'paperback', 'edition', 'September', 'SPECTRA', 'and', 'the', 'portrayal', 'of', 'a', 'boxed', 's', 'are', 'trademarks', 'of', 'Bantam', 'Books', 'a', 'division', 'of', 'Random', 'House', 'Inc', 'All', 'rights', 'reserved'], ['Copyright', 'by', 'George', 'R', 'R', 'Martin'], ['Maps', 'by', 'James', 'Sinclair'], ['Heraldic', 'crest', 'by', 'Virginia', 'Norey'], ['Library', 'of', 'Congress', 'Catalog', 'Card', 'Number'], ['No', 'part', 'of', 'this', 'book', 'may', 'be', 'reproduced', 'or', 'transmitted', 'in', 'any', 'form', 'or', 'by', 'any', 'means', 'electronic', 'or', 'mechanical', 'including', 'photocopying', 'recording', 'or', 'by', 'any', 'informa

In [44]:
token_count = sum(len(sentence) for sentence in sentences)
print('The book corpus contains {0} tokens '.format(token_count))

The book corpus contains 1818103 tokens 


In [45]:
num_features = 300

min_word_count = 3

num_workers = multiprocessing.cpu_count()

# Context window lenght
context = 7

# Downsampling setting for frequent words
downsampling = 1e-3

# seed for RNG to make to make result reproducable
seed = 1

In [47]:
thronestovec = wtv.Word2Vec(sg = 1,
                           seed = seed,
                           workers = num_workers,
                           size = num_features,
                           min_count = min_word_count,
                           window = context,
                           sample = downsampling
                           )

In [ ]:
# building the model
thronestovec.build_vocab(sentences)

In [60]:
# training the model
thronestovec.train(sentences, total_examples=thronestovec.corpus_count, epochs= 2)

(2808697, 3636206)

In [61]:
# making directory to save the model
if not os.path.exists('trained'):
    os.makedirs('trained')

In [62]:
# saving the model
thronestovec.save(os.path.join('trained','thronestovec.w2v'))

In [63]:
# Explore the trained model
thronestovec = wtv.Word2Vec.load(os.path.join('trained','thronestovec.w2v'))

In [71]:
# Compressing the whole three dimension vector to a 2D space
tsne = sklearn.manifold.TSNE(n_components = 2, random_state = 0)

In [79]:
all_words_vectors_matrix = thronestovec.syn1neg

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn1neg` (Attribute will be removed in 4.0.0, use self.trainables.syn1neg instead).
  """Entry point for launching an IPython kernel.


In [80]:
all_words_vectors_matrix_2d = tsne.fit_transform(all_words_vectors_matrix)
